In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import seaborn as sns
from speclearn.deep_learning.cluster import cluster_with_kmeans
from speclearn.deep_learning.model_utils import (get_colorbar,
                                                 load_beta_VAE_model)
from speclearn.deep_learning.predict import (get_full_data,
                                             predict_minerals,
                                             process_full_map, read_area)
from speclearn.deep_learning.model_utils import (get_colorbar,
                                                 load_beta_VAE_model)
from speclearn.deep_learning.predict import (get_full_data,
                                             process_full_map, read_area)
from speclearn.io.data.aoi import get_full_map_aoi
from speclearn.plot.map import *
from speclearn.tools.cache import check_file
from speclearn.tools.data_tools import *
from pysptools.spectro import FeaturesConvexHullQuotient
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
local_wavelength = select_wavelength(s_0=0, s_1=-12)
import datetime

print('Current time: ', datetime.datetime.now())
figure_dir = '/home/freya/Documents/figures/cluster_maps/'
from speclearn.tools.constants import *
import os

sns.set_style('whitegrid')
sns.set_context('notebook')

In [ ]:
#df = pd.read_pickle('/home/freya/Documents/Code/cache/band_features.pkl')

In [ ]:
k = 5
crs = False
area = 'large'
norm = False
full=False
if full:
    full_name='_full'
else:
    full_name=''

clist, cmap = get_colorbar(k)

aois = get_full_map_aoi(d_long=0.05, d_lat=0.05, step_size=10)
model, model_name = load_beta_VAE_model(crs=crs, norm=norm)
print('model:', model_name)

In [ ]:
crs = False
norm = False

data_2d_full, coord_full = get_full_data(aois, crs=crs, periods=[], norm=norm)
data_2d_full[np.fabs(data_2d_full) == 10] = float('NaN')

In [5]:
if check_file(os.path.join(CACHE_CLUSTER, f'{model_name}_{k}_cluster_2d_70_lat{full_name}.npy')):
    cluster_2d = np.load(os.path.join(CACHE_CLUSTER, f'{model_name}_{k}_cluster_2d_70_lat{full_name}.npy'))
    cluster_2d_s = np.load(os.path.join(CACHE_CLUSTER, f'{model_name}_{k}_cluster_2d_50_lat_south{full_name}.npy'))
    cluster_2d_n = np.load(os.path.join(CACHE_CLUSTER, f'{model_name}_{k}_cluster_2d_50_lat_north{full_name}.npy'))

cluster_2d_s = cluster_2d_s[:,0:400]
cluster_2d_n = cluster_2d_n[:,200:]

cluster_2d_full = np.concatenate([cluster_2d_s, cluster_2d, cluster_2d_n], axis=1)

In [6]:
def _is_band_none(band_1, band_2):
    if band_1 is None and band_2 is None:
        return [], [], [], []
    elif band_2 is None:
        selected_features = [band_1]
    elif band_1 is None:
        selected_features = [band_2]
    else:
        selected_features = [band_1, band_2]
    return selected_features

In [58]:
def get_band_features(features, band_1_range = [750, 1250], band_2_range = [1250, 2500]
):
    features_numbers = np.linspace(0, features.get_number_of_kept_features()-1,features.get_number_of_kept_features(), dtype=int)
    
    wl =  np.array([features.get_absorbtion_wavelength(i) for i in features_numbers])
    if len(features_numbers) == 0:
        return [], [], [], []
    
    depths = np.array([features.get_absorbtion_depth(i) for i in features_numbers])

    band_1_index = (wl>=band_1_range[0]) & (wl < band_1_range[1])
    band_1_candidates = features_numbers[band_1_index]
    band_1_candidate_depths = depths[band_1_index]
    band_1 = band_1_candidates[np.argmin(band_1_candidate_depths)] if band_1_candidates.any() else None
    
    band_2_index = (wl>=band_2_range[0]) & (wl < band_2_range[1])
    band_2_candidates = features_numbers[band_2_index]
    band_2_candidate_depths = depths[band_2_index]
    band_2 = band_2_candidates[np.argmin(band_2_candidate_depths)] if band_2_candidate_depths.any() else None

    selected_features = _is_band_none(band_1, band_2)
    if (len(selected_features) > 2):
        return selected_features
    
    selected_feature_numbers = [features_numbers[i] for i in selected_features]
    selected_features_data = [(features.get_absorbtion_depth(i),
                                features.get_absorbtion_wavelength(i),
                                features.get_area(i),
                                features.get_full_width_at_half_maximum(i)) for i in selected_feature_numbers]

    depths, absorbtion_wavelengths, areas, fwhm = zip(*selected_features_data)

    return depths, absorbtion_wavelengths, areas, fwhm

file_path = f'/home/freya/Documents/Code/cache/band_features_{model_name}.pkl'

if os.path.exists(file_path):
    df = pd.read_pickle(file_path)
else:    
    depths_1, absorbtion_wavelengths_1, areas_1, fwhm_1 = [], [], [], []
    depths_2, absorbtion_wavelengths_2, areas_2, fwhm_2 = [], [], [], []
    cluster, cluster_second = [], []

    for n_long in range(0, data_2d_full.shape[0]):
        for n_lat in range(0, data_2d_full.shape[1]):
            try:
                features = FeaturesConvexHullQuotient((data_2d_full[n_long, n_lat, :] + 1e-12).tolist(),
                                                      local_wavelength.tolist(),
                                                      normalize=False,
                                                      baseline=0.99)
            except Exception as e:
                # print(f'Error: {e}')
                continue

            if features.get_number_of_kept_features() > 0:
                depths, absorbtion_wavelengths, areas, fwhm = get_band_features(features, band_1_range = [500, 1250], band_2_range = [1250, 2500])

                cluster.append(cluster_2d_full[n_long, n_lat] + 1)
                features_numbers = np.arange(features.get_number_of_kept_features())
                
                if len(depths) == 0:
                    cluster_second.append(cluster_2d_full[n_long, n_lat] + 1)
                    depths_2.append(-1.)
                    absorbtion_wavelengths_2.append(-1.)
                    areas_2.append(-1.)
                    fwhm_2.append(-1.)               
                    
                    depths_1.append(-1.)
                    absorbtion_wavelengths_1.append(-1.)
                    areas_1.append(-1.)
                    fwhm_1.append(-1.)                         
                    continue

                if len(depths) > 1:
                    cluster_second.append(cluster_2d_full[n_long, n_lat] + 1)
                    depths_2.append(1 - depths[1])
                    absorbtion_wavelengths_2.append(absorbtion_wavelengths[1])
                    areas_2.append(areas[1])
                    fwhm_2.append(fwhm[1])
                else:
                    cluster_second.append(cluster_2d_full[n_long, n_lat] + 1)
                    depths_2.append(-1.)
                    absorbtion_wavelengths_2.append(-1.)
                    areas_2.append(-1.)
                    fwhm_2.append(-1.)
                

                depths_1.append(1 - depths[0])
                absorbtion_wavelengths_1.append(absorbtion_wavelengths[0])
                areas_1.append(areas[0])
                fwhm_1.append(fwhm[0])
            else:
                continue

    df = pd.DataFrame(columns=['Absorption wavelength', 'FWHM', 'Feature'])
    df_secondary = pd.DataFrame(columns=['Absorption wavelength', 'FWHM', 'Feature'])

    df['Depth'] = depths_1
    df['Absorption wavelength'] = absorbtion_wavelengths_1
    df['Area'] = areas_1
    df['FWHM'] = fwhm_1
    df['Feature'] = 'Band 1'
    df['Cluster'] = cluster

    if len(absorbtion_wavelengths) > 1:
        df_secondary['Depth'] = depths_2
        df_secondary['Absorption wavelength'] = absorbtion_wavelengths_2
        df_secondary['Area'] = areas_2
        df_secondary['FWHM'] = fwhm_2
        df_secondary['Feature'] = 'Band 2'
        df_secondary['Cluster'] = cluster_second

        df = df.append(df_secondary, ignore_index=True)

    df['Cluster'] = df['Cluster'].astype('int')
    pd.to_pickle(df, file_path)

In [ ]:
import matplotlib
a = ['#432e6b','#4580ba', '#b3b3b3', '#7cd250', '#fbeb37'] #fced69
cmap = matplotlib.colors.ListedColormap(a, "")
cmap.set_under('black')
cmap

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(15,3), sharex=False, sharey=False, gridspec_kw={'wspace': 0.2,'width_ratios': [2, 1, 1]})

sns.kdeplot(df['Absorption wavelength'], shade=False, shade_lowest=True, ax=ax[0],hue=df['Cluster'],palette=cmap, bw_adjust=1.2, common_norm=False)

#sns.kdeplot(df['FWHM'], shade=True, shade_lowest=False, ax=ax[1],hue=df['Cluster'],palette=cmap, bw_adjust=1.5,common_norm=False)

sns.kdeplot(data=df[df['Feature'] == 'Band 1'], x='Depth', shade=False, shade_lowest=True, ax=ax[1],hue='Cluster',palette=cmap, bw_adjust=1.5,common_norm=False)

sns.kdeplot(data=df[df['Feature'] == 'Band 2'], x='Depth', shade=False, shade_lowest=True, ax=ax[2],hue='Cluster',palette=cmap, bw_adjust=1.5,common_norm=False)

#i.set_title(f'Cluster {c+1}')

ax[0].set_xlim([500,2500])
ax[1].set_xlim([0,0.35])
ax[2].set_xlim([0,0.35])

ax[0].set_xlabel('Absorption wavelength [nm]')
ax[1].set_xlabel('Depth at Band I')
ax[2].set_xlabel('Depth at Band II')

#i.set_ylim([0,0.0055])
plt.savefig(os.path.join(figure_dir, f'absorption_{model_name}_{k}{full_name}_band_ranges.png'), dpi=300, bbox_inches='tight')
plt.show()

In [59]:
df.loc[df['Absorption wavelength']<1250.0, 'Feature'] = 'Band 1'
df.loc[df['Absorption wavelength']>1250.0, 'Feature'] = 'Band 2'

In [ ]:
sns.kdeplot(df['Absorption wavelength'], shade=False, shade_lowest=True, hue=df['Cluster'],palette=cmap, bw_adjust=1.2, common_norm=False)
plt.show()

In [ ]:
sns.kdeplot(df[df['Feature']=='Band 1']['Absorption wavelength'], shade=False, shade_lowest=True, hue=df['Cluster'],palette=cmap, common_norm=False,bw_adjust=30)
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=5, nrows=1, figsize=(15,3), sharex=True, sharey=True)
for c, i in enumerate(ax.ravel()):
    _df = df[df['Cluster']==c+1]
    #_df = _df[_df['Feature']=='Band 1']
    
    sns.histplot(_df['Absorption wavelength'], ax=i, edgecolor="None", alpha=0.65,stat='density',binwidth=100, color=clist[c])
    sns.kdeplot(_df['Absorption wavelength'], shade=False, shade_lowest=False, ax=i, c=clist[c], bw_adjust=1.5)
    i.set_title(f'Cluster {c+1}')
    #i.set_xlim([500,2500])

In [ ]:
fig, ax = plt.subplots(ncols=5, nrows=1, figsize=(15,3), sharex=True, sharey=True)
for c, i in enumerate(ax.ravel()):
    _df = df[df['Cluster']==c+1]
    _df = _df[_df['Feature']=='Band 1']
    
    sns.histplot(_df['Absorption wavelength'], ax=i, edgecolor="None", alpha=0.65,stat='density',binwidth=100, color=clist[c])
    sns.kdeplot(_df['Absorption wavelength'], shade=False, shade_lowest=False, ax=i, c=clist[c], bw_adjust=10.5)
    i.set_title(f'Cluster {c+1}')
    #i.set_xlim([500,2500])

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Define the number of clusters and their respective colors
num_clusters = 5
clist = ['blue', 'green', 'red', 'orange', 'purple']  # Define your colors

fig, ax = plt.subplots(ncols=num_clusters, nrows=1, figsize=(15,3), sharex=True, sharey=True)

for c, i in enumerate(ax.ravel()):
    _df = df[df['Cluster'] == c+1]
    _df = _df[_df['Feature'] == 'Band 1']

    sns.histplot(_df['Absorption wavelength'], ax=i, edgecolor="None", alpha=0.65, stat='density', binwidth=100, color=clist[c])
    kde = sns.kdeplot(_df['Absorption wavelength'], shade=False, shade_lowest=False, ax=i, c=clist[c], bw_adjust=10.5)
    i.set_title(f'Cluster {c+1}')

    # Get x and y values of the KDE plot
    x_kde, y_kde = kde.get_lines()[0].get_data()

    # Calculate cumulative density function (CDF)
    cdf = np.cumsum(y_kde) / np.sum(y_kde)

    # Find the x-value where CDF is closest to 0.5
    median_index = np.argmin(np.abs(cdf - 0.5))
    median = x_kde[median_index]

    print(f"Median for Cluster {c+1}: {median}")

plt.show()

In [ ]:
np.around(df[df['Feature']=='Band 1'].groupby('Cluster').mean(),3)

In [ ]:
np.around(df[df['Feature']=='Band 1'].groupby('Cluster').median(),3)

In [ ]:
np.around(df[df['Feature']=='Band 2'].groupby('Cluster').mean(),3)

In [ ]:
np.around(df[df['Feature']=='Band 2'].groupby('Cluster').median(),5)